In [64]:
import os
import sys
import PIL
import requests
import PIL.Image as Image
from tqdm import tqdm
import numpy as np
import pandas as pd

In [49]:
names =[]


for i in os.listdir('Subreddit Posts'):
    if i.endswith('.csv'):
        df=pd.read_csv(f'Subreddit Posts/{i}')
        images=df[df['url'].apply(lambda x: any([x.endswith(img_end) for img_end in ['jpg','jpeg','png']]))]
        images= images[~images['over_18']]
        
        def in_dataset(x):
            return os.path.exists(f"dataset_final/Weapon/{x.replace('jpeg','jpg').replace('.png','jpg')}") or os.path.exists(f"dataset_final/People/{x.replace('jpeg','jpg').replace('.png','jpg')}")
        
       
        images['filenames'] = images['url'].apply(lambda x: x.split('/')[-1])
        images=images[images['filenames'].apply(in_dataset)]
        images['subreddit']=i[:-4]
        
        names.append(images)
            

/var/folders/zx/_x1xg_4n2hj8d_6dx7nmrm5m0000gn/T/ipykernel_25673/743759679.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(f'Subreddit Posts/{i}')
/var/folders/zx/_x1xg_4n2hj8d_6dx7nmrm5m0000gn/T/ipykernel_25673/743759679.py:6: DtypeWarning: Columns (8,19,80,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(f'Subreddit Posts/{i}')
/var/folders/zx/_x1xg_4n2hj8d_6dx7nmrm5m0000gn/T/ipykernel_25673/743759679.py:6: DtypeWarning: Columns (0,1,4,5,6,8,9,10,11,12,13,14,18,19,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,39,43,45,50,51,52,56,62,65,66,68,69,72,73,74,75,80,86,88,90,91,92,93,97,98,99,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(f'Subreddit Posts/{i}')
/var/folders/zx/_x1xg_4n2hj8d_6dx7nmrm5m0000gn/T/ipykernel_25673/743759679.py:6: DtypeWarning: Columns (95,102) have mixed types. Specify dtype option 

In [74]:

working=pd.concat(names)

folder_labels = {'Idiotswithguns':'Weapon',
                'TheWayWeWere':'People',
                 'ChicksWithGuns': 'Weapon',
                 'Weapons': 'Weapon',
                 'portraits': 'People',
                 'guns': 'Weapon',
                 'Military': 'Weapon',
                 'pics': 'Weapon',
                 'Shooting': 'Weapon',
                 'Guns_Guns_Guns': 'Weapon',
                 'airsoft': 'Weapon',
                 'Shotguns': 'Weapon'
                }
working['label']=working['subreddit'].map(folder_labels)

/var/folders/zx/_x1xg_4n2hj8d_6dx7nmrm5m0000gn/T/ipykernel_25673/471567472.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  working['label']=working['subreddit'].map(folder_labels)


In [75]:
working[['label','url']].to_csv('data_download.csv',index=False)


In [76]:
import os
import sys
import shutil
import PIL
import requests
import PIL.Image as Image
from tqdm import tqdm
import numpy as np
import pandas as pd

def load_image(filename):
    return Image.open(filename).convert('RGB') #this conversion is crucial as PIL will open png files with either 3 or 4 channels depending on encoding

def center_crop(img):
    '''Algorithm derived from nvidia preprocessor'''
    crop = np.min(img.shape[:2])
    img = img[(img.shape[0]-crop)//2:(img.shape[0]+crop)//2,(img.shape[1]-crop)//2:(img.shape[1]+crop)//2]
    img = Image.fromarray(img,'RGB')
    return img

def resize_and_crop(fp,width,height):
    img=np.asarray(load_image(fp),dtype=np.uint8)
    img=center_crop(img)
    
#     if img.size[1]<height or img.size[0]<width:
#         print('here')
#         print(fp)
    
    img=img.resize((width,height),Image.LANCZOS)
    img=np.array(img)
    
    canvas = np.zeros([width,width,3],dtype=np.uint8)
    canvas[(width-height)//2: (width+height)//2,:]=img
    return Image.fromarray(canvas )

        #img.save(f'filen.jpg')


def download_images(reddit_csv):
    print('Downloading and Cropping Images')
    fp ='Data'
    fpt = os.path.join('Data','temp')
    if not os.path.exists(fp):
        os.mkdir(fp)
    os.mkdir(fpt)
    fpw= os.path.join(fp,'Weapon')
    fpp =os.path.join(fp,'People')
    os.mkdir(fpw)
    os.mkdir(fpp)
    df=pd.read_csv(reddit_csv)
    
    post_fix = {'Success':0,'Failed':0}

    
    pbar = tqdm(df.iterrows(),total=len(images))
    
    for index, row in pbar:
        image_url = row.url
        filename = os.path.join(fpt,row['url'].split('/')[-1])
        label = row.label
        filename=os.path.join(fpt,row['url'].split('/')[-1])
        
        try:
            r= requests.get(image_url,stream=True)
        except:
            post_fix['Failed']+=1 
            continue
        if r.status_code==200:
            r.raw.decode_content = True
            with open(filename,'wb') as f:
                shutil.copyfileobj(r.raw,f)
            post_fix['Success']+=1
        else:
            post_fix['Failed']+=1
        
        ##Crop
        canvas=resize_and_crop(filename,256,256)
        if label =='People':
            fn2 =os.path.join(fpp,row['url'].split('/')[-1])
        if label =='Weapon':
             fn2 =os.path.join(fpw,row['url'].split('/')[-1])
        canvas.save(fn2)
        os.remove(filename)
        pbar.set_postfix(post_fix)
    pbar.close()
    os.rmdir(fpt)
    

In [73]:
download_images('data_download.csv')

0it [00:00, ?it/s]/var/folders/zx/_x1xg_4n2hj8d_6dx7nmrm5m0000gn/T/ipykernel_25673/3467461753.py:29: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img=img.resize((width,height),Image.LANCZOS)
1658it [09:13,  2.99it/s, Success=1657, Failed=1]
Exception ignored in: <function tqdm.__del__ at 0x113834310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tqdm/std.py", line 1161, in __del__
KeyboardInterrupt: 


KeyboardInterrupt: 

In [55]:
pd.read_csv('data_download.csv')

,label,url
0,Weapon,https://i.redd.it/972cb5iyksw61.jpg
1,Weapon,https://i.redd.it/zmjqk540khw81.jpg
2,Weapon,https://i.redd.it/ykod4voz0tv81.jpg
3,Weapon,https://i.redd.it/p7g37ufbfdv81.jpg
4,Weapon,https://i.redd.it/uohp2v6f6zu81.jpg
...,...,...
4590,Weapon,https://i.redd.it/ut3u18s8jbt81.jpg
4591,Weapon,https://i.redd.it/x7ggkud2uat81.jpg
4592,Weapon,https://i.redd.it/1hripmn3l8t81.jpg
4593,Weapon,https://i.redd.it/2rhasjywo5t81.jpg
